In [2]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [3]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [4]:

X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)


In [7]:
import fresh.preproc.v2 as pv2
import ipdb


In [ ]:
reload(pv2)
# wow kernel keeps dying here .. guess got to baatch the preprocessing.
#with ipdb.launch_ipdb_on_exception():
X_transformed, y_enc, proc_dict = ipdb.runcall(pv2.preprocess,
        X, y, neighborhoods, labeled=True)

> /opt/program/fresh/preproc/v2.py(10)preprocess()
      9 def preprocess(X, y, neighborhoods, labeled, proc_bundle=None):
---> 10     num_rows = X.shape[0]
     11 

ipdb> n
> /opt/program/fresh/preproc/v2.py(12)preprocess()
     11 
---> 12     if proc_bundle:
     13         X_transformed, y_enc = xform(proc_bundle, X, y)

ipdb> 
> /opt/program/fresh/preproc/v2.py(17)preprocess()
     16         'usertype'  # 'Customer', 'Subscriber'
---> 17         genders = [0, 1, 2]
     18         user_types = ['Subscriber', 'Customer']

ipdb> n
> /opt/program/fresh/preproc/v2.py(18)preprocess()
     17         genders = [0, 1, 2]
---> 18         user_types = ['Subscriber', 'Customer']
     19         time_of_day = [0, 1, 2, 3, 4]

ipdb> 
> /opt/program/fresh/preproc/v2.py(19)preprocess()
     18         user_types = ['Subscriber', 'Customer']
---> 19         time_of_day = [0, 1, 2, 3, 4]
     20         features = ['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday', ]

ipdb> 


ipdb> p fu
*** NameError: name 'fu' is not defined
ipdb> s
--Call--
> /opt/program/fresh/preproc/v2.py(38)xform()
     37 
---> 38 def xform(proc_bundle, X, y=None):
     39     '''Apply preprocessing to X, y.  '''

ipdb> n
> /opt/program/fresh/preproc/v2.py(43)xform()
     42     #
---> 43     num_rows = X.shape[0]
     44     X_transformed = np.hstack((

ipdb> n
> /opt/program/fresh/preproc/v2.py(44)xform()
     43     num_rows = X.shape[0]
---> 44     X_transformed = np.hstack((
     45         proc_bundle['enc'].transform(X[:, :3]).toarray(),

ipdb> l
     39     '''Apply preprocessing to X, y.  '''
     40     # TODO ... also the y_enc part,
     41     # , for which need to also handle missing..
     42     #
     43     num_rows = X.shape[0]
---> 44     X_transformed = np.hstack((
     45         proc_bundle['enc'].transform(X[:, :3]).toarray(),
     46         np.resize(
     47             proc_bundle['usertype_le'].transform(X[:, 3]),
     48             (num_rows, 1)
     49

In [8]:
X_transformed[0]

array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 1.0, 0, 1], dtype=object)

In [2]:
#fu.get_my_memory()#
!top

=top - 13:19:19 up 6 days,  4:33,  0 users,  load average: 0.59, 0.29, 0.14
Tasks:   5 total,   1 running,   4 sleeping,   0 stopped,   0 zombie
%Cpu(s):  7.1 us,  0.6 sy,  0.0 ni, 92.1 id,  0.2 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem :  2046844 total,  1820120 free,   111344 used,   115380 buff/cache
KiB Swap:  1048572 total,   776864 free,   271708 used.  1791560 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
 1296 root      20   0  562408  52916  13632 S   6.7  2.6   0:00.30 python      
    1 root      20   0   18236     40      0 S   0.0  0.0   0:00.22 bash        
 1053 root      20   0  558948  19272   6204 S   0.0  0.9   0:18.98 jupyter-no+ 
 1310 root      20   0    4500    748    676 S   0.0  0.0   0:00.52 sh          
 1311 root      20   0   36852   3076   2668 R   0.0  0.2   0:00.00 top         
>
